# SF Affordable Housing

### By Irene Casado Sanchez

The purpose of this notebook is to explore the SF Affordable Housing Pipeline. 

Rising prices in the city of San Francisco have forced thousands of residents to leave the city. To try to counter this trend, the City and County of San Francisco committed in its 2014 Housing Element to "permanently prioritize affordable housing." 

Part of our notebook focuses on the evolution of the affordable housing since this commitment was publicly advertised. 

Main questions:
- How many affordable units have been built in the last five years?
- How many units are going to be built in the next years?
- What is the trend over time in affordable housing construction?
- Where in San Francisco are the most affordable units being built?
- Are the city's affordable construction objectives being reached?

Main fields used in this analysis:
- `project_status`
- `estimated_actual_construction_start_date`
- `estimated_construction_completion`
- `project_units`
- `affordable_units` 
- `per_affordable`
- `city_analysis_neighborhood`

### Data Description:

* **Main data source**: Affordable Housing Pipeline, SF Data (updated February 7, 2022, url: https://data.sfgov.org/Housing-and-Buildings/Affordable-Housing-Pipeline/aaxw-2cb8).
* **Data details**: "The projects listed are in the process of development--or are anticipated to be developed--in partnership with non-profit or for-profit developers [...] The Affordable Housing Pipeline also includes housing units produced by private developers through the Inclusionary Affordable Housing Program.""
* **Time range**: Data reflects all projects as of December 31, 2021 – the oldest project included in the database is dated July 8, 2004.
* **Total values**: 580

*****
 
* **Additional data source**: SF 2014 Housing Element Table – PART II: OBJECTIVES & POLICIES, url: https://generalplan.sfplanning.org/2014HousingElement-AllParts_ADOPTED_web.pdf.
* **Data details**: The dataset is based on a table that appears in the report.
* **Time range**:  The dataset listed protential projects between 2014 and 2022.
* **Total values**: 17

### Data Limitations:

* Missing values: 30 records missing `estimated_actual_construction_start_date` – Affordable Housing Pipeline, SF Data.
* In two rows, the `estimated_construction_completion` is prior to `estimateda_actual_construction_start_date` – Affordable Housing Pipeline, SF Data.
* The estimations of the **SF 2014 Housing Element** only include **8 SF neighborhoods**.
* These projections are limited to the period between **2014** and **2022**. 

### Key Findings:

* In the last five years, **2,789** affordable units have been built.
* There is a delay in the construction of over **2,000 affordable units**. 
* More than **2,500 affordable units** are expected to be completed between 2022 and 2024. 
* Projects completed after the advent of the pandemic required an average of **420** more days of construction than those completed previously. 
* The neighborhoods where most construction has taken place in the last ten years are **South of Market** and **Financial District**, in these areas the percentage of **affordable units** does not exceed **26%**. 
* **Mission** and **Bayview Hunters Point** neighborhoods have experienced a significant increase in the number of units built; in these cases, the percentage of affordable units is **45%** and **73%**, respectively. 
* The SF city's forecast is far from the actual figures. The largest differences between the city's estimates and the actual figures appear in the following neighborhoods: **Bayview Hunters Point, South of Market and Financial District**.





## Configuration:
Let's begin by importing the Python tools necessary for the job.

In [813]:
import pandas as pd
import altair as alt
import numpy as np
import datetime
import os
from vega_datasets import data
pd.set_option('display.max_columns', None)

Read the Affordable Housing Pipelinethe Dataset. This version of the database was downloaded on February 28, 2022.

In [814]:
data_dir = os.environ["DATA_DIR"]
raw_data = data_dir + "/raw/"
housing_file = os.path.join(raw_data, 'Affordable_Housing_Pipeline.csv')
housing_raw = pd.read_csv(housing_file)

### Getting acquainted with the data

Let's get acquainted with the columns using `info()`.

In [815]:
housing_raw.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 71 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Project ID                                580 non-null    object 
 1   Project Status                            580 non-null    object 
 2   Project Name                              580 non-null    object 
 3   Street Number                             551 non-null    object 
 4   Street Name                               580 non-null    object 
 5   Street Type                               541 non-null    object 
 6   Zip Code                                  580 non-null    int64  
 7   Block                                     553 non-null    object 
 8   Lot                                       551 non-null    object 
 9   Supervisor District                       580 non-null    int64  
 10  City Analysis Neighborhood            

Here, we prepare the data to simplify downstream code:

In [816]:
# Punctuation configuration

pd.options.display.float_format = '{:,.0f}'.format

In [817]:
# Removal of whitespace and hidden characters

housing_raw.columns = housing_raw.columns.str.replace(' ', '_').str.lower().str.replace('/', '_').str.replace('%', 'per')  # Reformat columns name

In [818]:
# Create a copy of database with new name and reformated columns

housing = housing_raw.copy() 

### Data vetting
Before diving into analysis, we need to do some preliminary data quality checks. Here are some key questions to always ask:

* Are there any missing or malformed values?
* Are there any unexpected values?
* Are there duplicate records?

#### Date vetting

In [819]:
# Checking potential empty rows in estimated_actual_construction_start_date label

housing.estimated_actual_construction_start_date.isna().value_counts()

False    550
True      30
Name: estimated_actual_construction_start_date, dtype: int64

In [820]:
housing.estimated_construction_completion.isna().value_counts()

False    550
True      30
Name: estimated_construction_completion, dtype: int64

In [821]:
housing[['estimated_construction_completion', 'estimated_actual_construction_start_date']].describe()

,estimated_construction_completion,estimated_actual_construction_start_date
count,550,550
unique,463,433
top,06/29/2019,11/01/2015
freq,9,10


Thirty records in our overall data set are missing `estimated_actual_construction_start_date` label. This number likely won't affect statistics on the full data set, but it is important to keep it in mind.

In order to be able to permorm our analysis, we need to delete those empty fielfs. 

In [822]:
# View the records that we are going to drop

housing[housing.estimated_construction_completion.isna()].head(3)

,project_id,project_status,project_name,street_number_,street_name,street_type,zip_code,block,lot,supervisor_district,city_analysis_neighborhood,planning_neighborhood,lead_agency,program_area,project_area,project_type,housing_tenure,ssp_acquisition_date,issuance_of_notice_to_proceed,issuance_of_building_permit,issuance_of_first_construction_document,estimated_actual_construction_start_date,estimated_construction_completion,project_lead_sponsor,project_co-sponsor,project_owner,planning__case_number,property_informaiton_map_link,planning_entitlements,entitlement_approval,section_415_declaration,building_permit_number,project_units,affordable_units,market_rate_units,per_affordable,sro_units,studio_units,1bd_units,2bd_units,3bd_units,4bd_units,5+_bd_units,mobility_units,manager_units,manager_unit(s)_type,family_units,senior_units,tay_units,homeless_units,disabled_units,losp_units,public_housing_replacement_units,20per_ami,30per_ami,40per_ami,50per_ami,55per_ami,60per_ami,80per_ami,90per_ami,100per_ami,105per_ami,110per_ami,120per_ami,130per_ami,150per_ami,ami_undeclared,latitude,longitude,location
30,1984-010,(6) Complete,South Beach Marina Apartments,2,Townsend,St,94107,3789,"028, 029",6,Financial District/South Beach,South of Market,MOHCD,Bond Financed Multifamily Housing,NaN,Rehabilitation,Rental,NaN,NaN,NaN,NaN,NaN,NaN,"South Beach Marina , Inc.",NaN,"South Beach Marina , Inc.",Not Applicable,NaN,NaN,NaN,NaN,NaN,414,101,313,24,NaN,NaN,53,48,NaN,NaN,NaN,NaN,NaN,NaN,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101,NaN,NaN,NaN,38,-122,"(37.78240667, -122.389807907999)"
44,2020-016,(1) Preliminary Phase,71 Boardman,71,Boardman,Pl,94103,3779,84,6,South of Market,South of Market,MOHCD,Multifamily Rental Program,NaN,New Construction,Rental,NaN,NaN,NaN,NaN,NaN,NaN,The John Stewart Company,Conard House,NaN,Not Yet Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.77448, -122.40235)"
63,2006-002,(7) On-Hold / Planning Approval Expired,Midtown Park Apartments,1415,Scott,St,94115,1099,31,5,Western Addition,Western Addition,MOHCD,Multifamily Rental Program,NaN,Rehabilitation,Rental,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Yet Available,NaN,2014.0563R,NaN,NaN,NaN,139,139,NaN,100,NaN,NaN,23,55,61,NaN,NaN,NaN,NaN,NaN,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,NaN,10,15,15,10,NaN,NaN,10,NaN,1,13,38,-122,"(37.783005259, -122.438499577999)"


In [823]:
# Remove the empty rows

housing = housing[housing.estimated_actual_construction_start_date.isna() == False] 

Part of our analysis focuses on the construction of affordable housing over time. In order to make different calculations and graphs, we need to extract the year from `estimateda_actual_construction_start_date` and
`estimated_construction_completion`.

In [824]:
# Creating a year column based on the year_construction_start and year_construction_completion

housing['year_construction_start'] = pd.DatetimeIndex(housing['estimated_actual_construction_start_date']).year.astype(int)
housing['year_construction_completion'] = pd.DatetimeIndex(housing['estimated_construction_completion']).year.astype(int)
housing.reset_index()
housing.head(3)

,project_id,project_status,project_name,street_number_,street_name,street_type,zip_code,block,lot,supervisor_district,city_analysis_neighborhood,planning_neighborhood,lead_agency,program_area,project_area,project_type,housing_tenure,ssp_acquisition_date,issuance_of_notice_to_proceed,issuance_of_building_permit,issuance_of_first_construction_document,estimated_actual_construction_start_date,estimated_construction_completion,project_lead_sponsor,project_co-sponsor,project_owner,planning__case_number,property_informaiton_map_link,planning_entitlements,entitlement_approval,section_415_declaration,building_permit_number,project_units,affordable_units,market_rate_units,per_affordable,sro_units,studio_units,1bd_units,2bd_units,3bd_units,4bd_units,5+_bd_units,mobility_units,manager_units,manager_unit(s)_type,family_units,senior_units,tay_units,homeless_units,disabled_units,losp_units,public_housing_replacement_units,20per_ami,30per_ami,40per_ami,50per_ami,55per_ami,60per_ami,80per_ami,90per_ami,100per_ami,105per_ami,110per_ami,120per_ami,130per_ami,150per_ami,ami_undeclared,latitude,longitude,location,year_construction_start,year_construction_completion
0,2008-009,(6) Complete,"HPSY I, Block 50",547,Donahue,St,94124,4591C,"265, 266",10,Bayview Hunters Point,Bayview,OCII,Limited Equity Program,HP Shipyard Phase 1,New Construction,Ownership,NaN,NaN,12/29/2009,05/10/2010,05/10/2010,04/27/2016,Lennar Urban,NaN,NaN,Not Applicable,NaN,NaN,NaN,NaN,200907223200,25,3,22,12,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.728381185, -122.370199597999)",2010,2016
1,2013-044,(6) Complete,Mission Dolores,1855,15th,St,94103,3556,276,8,Mission,Mission,MOHCD,Rental Assistance Demonstration Program (RAD),NaN,Rehabilitation,Rental,10/12/2016,11/07/2016,05/16/2016,11/03/2016,11/03/2016,09/19/2018,BRIDGE Housing Corporation,Mission Economic Development Agency,"Bridge Housing Ventures, Inc, MEDA Housing LLC",2015-012300PRJ,http://propertymap.sfplanning.org/?search=2015...,2015-012300ENV,NaN,NaN,201510301342,91,90,1,99,NaN,68,22,NaN,NaN,NaN,NaN,11,1,1bd,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.766017308, -122.425573258999)",2016,2018
2,2014-032,(6) Complete,Britton Courts (1250 Sunnydale Ave),171,Loehr,St,94134,6300,43,10,Visitacion Valley,Visitacion Valley,MOHCD,Bond Financed Multifamily Housing,NaN,Rehabilitation,Rental,NaN,12/01/2017,NaN,NaN,12/01/2017,12/01/2018,Mercy Housing California,NaN,"Mercy Housing California 74, LP",Not Applicable,NaN,NaN,NaN,NaN,NaN,92,91,1,99,NaN,NaN,NaN,62,21,8,NaN,NaN,1,2bd,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,NaN,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.711716304, -122.413358024999)",2017,2018


#### Project Status Vetting

One of our first steps involves a few quick checks on the `project_status`.

In [825]:
# Viewing description of project_status

housing.project_status.describe()

count              550
unique               8
top       (6) Complete
freq               261
Name: project_status, dtype: object

In [826]:
# Info project_status

housing.project_status.info()

<class 'pandas.core.series.Series'>
Int64Index: 550 entries, 0 to 579
Series name: project_status
Non-Null Count  Dtype 
--------------  ----- 
550 non-null    object
dtypes: object(1)
memory usage: 8.6+ KB


To facilitate future analysis, here we change the format of the needed rows. 

In [827]:
# Creating a lowercase version of the original project_status column

housing['project_status'] = housing['project_status'].str.lower().apply(lambda x: '_'.join(x.split(' ')[1:]))

In [828]:
# Viewing unique project_status

housing['project_status'].unique()

array(['complete', 'first_construction_document_issued',
       'preliminary_phase', 'predevelopment_feasibility',
       'site_work_permit_issued', 'design_with_entitlements_approved',
       'on-hold_/_planning_approval_expired', 'cancelled'], dtype=object)

#### City Analysis Neighborhood Vetting

Now, we are going to do exactly the same but with another category `city_analysis_neigborhood`.

In [829]:
# Viewing description of city_analysis_neighborhood

housing['city_analysis_neighborhood'].describe()

count         550
unique         35
top       Mission
freq           84
Name: city_analysis_neighborhood, dtype: object

In [830]:
# Info of city_analysis_neighborhood

housing.city_analysis_neighborhood.info()

<class 'pandas.core.series.Series'>
Int64Index: 550 entries, 0 to 579
Series name: city_analysis_neighborhood
Non-Null Count  Dtype 
--------------  ----- 
550 non-null    object
dtypes: object(1)
memory usage: 8.6+ KB


In [831]:
# Viewing unique city_analysis_neighborhood

housing['city_analysis_neighborhood'].unique()

array(['Bayview Hunters Point', 'Mission', 'Visitacion Valley',
       'Tenderloin', 'Marina', 'Bernal Heights', 'Inner Richmond',
       'Hayes Valley', 'Sunset/Parkside', 'South of Market', 'Chinatown',
       'Mission Bay', 'Lakeshore', 'Oceanview/Merced/Ingleside',
       'Financial District/South Beach', 'Portola', 'Lone Mountain/USF',
       'Potrero Hill', 'Outer Richmond', 'Pacific Heights', 'Nob Hill',
       'Presidio Heights', 'Western Addition', 'Castro/Upper Market',
       'West of Twin Peaks', 'Outer Mission', 'North Beach',
       'Inner Sunset', 'Russian Hill', 'Haight Ashbury', 'Japantown',
       'Excelsior', 'Treasure Island', 'Twin Peaks', 'Noe Valley'],
      dtype=object)

In [832]:
# Creating a lowercase version of the original city_analysis_neighborhood column

housing['city_analysis_neighborhood'] = housing['city_analysis_neighborhood'].str.replace(' ', '_').str.lower().str.replace('/', '_')

## A. Project Status Analysis

We have checked the quality of our database, drop empty rows and setup our data, then we can start our analysis. 

The first part of our analysis focuses on analyzing the status of the projects over time. 

Key data points: 
   * **2016** is the year with the **most affordable units completed**, followed by 2015 and 2017. 
   * **2020** is the year with the largest number of **first construction documents issued**, followed by 2021 and 2019. 
   * **2021** is the year with the largest number of affordable units with **site work permit issued**, followed by 2020 and 2019.
   * In the last five years, **2,789** affordable units have been built.

Key findings:

   * There is a delay in the construction of over **1,000 affordable units**. 56 units that were scheduled to be completed in 2019, 277 that were expected to be completed in 2020 and 683 that were expected to be completed in 2021 are still under construction. 
  * More than **2,500 affordable units** are expected to be completed between 2022 and 2024. 

In [833]:
# Exploring project_status

housing.project_status.value_counts()

complete                               261
first_construction_document_issued      97
design_with_entitlements_approved       88
site_work_permit_issued                 49
predevelopment_feasibility              34
preliminary_phase                       18
cancelled                                2
on-hold_/_planning_approval_expired      1
Name: project_status, dtype: int64

The best way to check the different trends by project status is to make a specific graph for each type of status.

First, let's calculate how many affordable units have been **completed** each year. To do this, we need to filter by `project_status` - `complete`.

In [834]:
# Create a table with completed units.

complete = housing[housing['project_status'] == 'complete'].groupby('year_construction_start').agg({'affordable_units':'sum', 'per_affordable': 'median'}).reset_index()
complete

,year_construction_start,affordable_units,per_affordable
0,2004,7,13
1,2007,5,13
2,2008,4,12
3,2010,3,12
4,2011,126,55
5,2012,678,14
6,2013,668,14
7,2014,774,12
8,2015,"2,343",20
9,2016,"3,141",98


In [835]:
# Create a chart

source = complete
alt.Chart(source).mark_bar(size=10).encode(
        x = "year_construction_start", 
        y= "affordable_units", 
).properties(width=350, title='Affordable Units Completed per Year')

alt.Chart(...)

Here, we prepare the data for the next chart: **"Affordable Units with First Construction Document per Year."**

We have to delete the empty fields [`issuance_of_first_construction_document`] in order to make our table and chart. The empty rows are not an error, but rather that the document has not been issued.

In [836]:
# Checking the empty fields and dropping them

housing.issuance_of_first_construction_document.isna().value_counts()
housing_first_document = housing[housing.issuance_of_first_construction_document.isna() == False]

In [837]:
# Create a new column with 'first_construction_document_year'

housing_first_document['first_construction_document_year'] = pd.DatetimeIndex(housing_first_document['issuance_of_first_construction_document']).year.astype(int)

/var/folders/5t/ptdbrrps2zzc0q0jvwkbsh980000gn/T/ipykernel_22109/2370009985.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_first_document['first_construction_document_year'] = pd.DatetimeIndex(housing_first_document['issuance_of_first_construction_document']).year.astype(int)


In [838]:
# Create a new table with the required fields for our chart

housing_first_document_year = housing_first_document[housing_first_document['project_status'] == 'first_construction_document_issued'].groupby('first_construction_document_year').agg({'affordable_units':'sum', 'per_affordable': 'median'}).reset_index()
housing_first_document_year

,first_construction_document_year,affordable_units,per_affordable
0,2014,17,12
1,2015,7,13
2,2016,0,0
3,2017,200,8
4,2018,176,0
5,2019,412,12
6,2020,960,18
7,2021,532,99


In [839]:
# Create a chart

source = housing_first_document_year
alt.Chart(source).mark_bar(size=10).encode(
        x = "first_construction_document_year", 
        y= "affordable_units", 
).properties(width=300, title='Affordable Units with First Construction Document per Year')

alt.Chart(...)

In [840]:
# Create a line chart

source = housing_first_document_year

alt.Chart(source).mark_line().encode(
    x='first_construction_document_year:O',
    y='affordable_units',
).properties(width=300, title='Affordable Units with First Construction Document Issued over Time')

alt.Chart(...)

The two charts above give us an idea of the time trends in obtaining the first construction document: we observe a significant increase in the granting of these permits between 2018 and 2020, and a significant drop in 2021. 


Exactly as in the previous element, here, we prepare the data for the next chart: **"Affordable Units with Building Permit Issued per Year."**

We have to delete the empty fields [`issuance_of_building_permit`] in order to make our table and chart. The empty rows are not an error, but rather that the document has not been issued. 

In [841]:
# Checking the empty fields and dropping them

housing.issuance_of_building_permit.isna().value_counts()
housing_building_permit = housing[housing.issuance_of_building_permit.isna() == False]

In [842]:
# Create a new column with 'building_permit_year'

housing_building_permit['building_permit_year'] = pd.DatetimeIndex(housing_building_permit['issuance_of_building_permit']).year.astype(int)

/var/folders/5t/ptdbrrps2zzc0q0jvwkbsh980000gn/T/ipykernel_22109/645646699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_building_permit['building_permit_year'] = pd.DatetimeIndex(housing_building_permit['issuance_of_building_permit']).year.astype(int)


In [843]:
# Create a new table with the required fields for our chart

housing_building_permit_year = housing_building_permit[housing_building_permit['project_status'] == 'site_work_permit_issued'].groupby('building_permit_year').agg({'affordable_units':'sum', 'per_affordable': 'median'}).reset_index()
housing_building_permit_year

,building_permit_year,affordable_units,per_affordable
0,2015,4,11
1,2016,21,11
2,2017,0,0
3,2018,130,14
4,2019,249,14
5,2020,320,14
6,2021,338,99


In [844]:
# Create a chart

source = housing_building_permit_year
alt.Chart(source).mark_bar(size=10).encode(
        x = "building_permit_year", 
        y= "affordable_units", 
).properties(width=300, title='Affordable Units with Building Permit Issued per Year')

alt.Chart(...)

In [845]:
# Create a line chart

source = housing_building_permit_year

alt.Chart(source).mark_line().encode(
    x='building_permit_year:O',
    y='affordable_units',
).properties(width=300, title='Affordable Units with Building Permit over Time')

alt.Chart(...)

The two charts above give us an idea of the time trends in the granting of building permits:
* We observe a significant increase between 2018 and 2019.
* It is noteworthy that in 2017 no permits were granted.

Next, let's calculate the **future construction of affordable units**. To do so, we are going to filter our data according to the `project _status`. Here we will only take into account projects that are currently under development:`first_construction_document_issued`, `design_with_entitlements_approved` and `site_work_permit_issued`.

In [846]:
### Filter the data by proyect_status and create a new table with the required fields

potential_future_units = housing.loc[(housing['project_status'] == 'first_construction_document_issued') | (housing['project_status'] == 'design_with_entitlements_approved') | (housing['project_status'] == 'site_work_permit_issued')].groupby('year_construction_completion').agg({'affordable_units':'sum'}).reset_index()
potential_future_units

,year_construction_completion,affordable_units
0,2019,317
1,2020,377
2,2021,"1,322"
3,2022,"2,442"
4,2023,"1,570"
5,2024,424
6,2025,146
7,2026,330


In [847]:
# Create a chart

source = potential_future_units
alt.Chart(source).mark_bar(size=10).encode(
        x = "year_construction_completion", 
        y= "affordable_units", 
).properties(width=250, title='Estimation of Future Affordable Units')

alt.Chart(...)

This table shows that: 
   * More than **2,400** affordable units will be completed in **2022**.
   * More than **1,500** affordable units will be completed in **2023**.
   * There is a **delay** in the construction of over **2,000** affordable units.

The purpose of the following graph is to visualize the trends of the different status over time: **"Construction Status Affordable Units Over Time"**

In [848]:
# Here I filter the 'status' that I need for my next chart

filter_status_per_year = housing[housing.project_status.isin(['complete', 'design_with_entitlements_approved', 'site_work_permit_issued', 'first_construction_document_issued']) == True]
status_per_year = filter_status_per_year.groupby(['project_status', 'year_construction_start']).affordable_units.sum().reset_index()

In [849]:
# Here I would like to create a grapg with the tendency of the for status

source = status_per_year

alt.Chart(source).mark_line().encode(
    x='year_construction_start:O',
    y='affordable_units',
    color='project_status').properties(width=600, title='Construction Status Affordable Units Over Time')

alt.Chart(...)

It is interesting to look at this graph for correlations. We see a significant drop in the number of units completed in 2016, and this may be related to trends in earlier phases of construction.

As the last element of our analysis focused on the status of the project, we will graph the **cumulative number of units completed over time**. 

In [850]:
# Grouping by year and calculation cumulative sum

complete = filter_data('complete')
complete_grouped = complete.groupby('year_construction_completion').affordable_units.sum().reset_index().sort_values('year_construction_completion')
complete_grouped['affordable_units_cumsum'] = complete_grouped.affordable_units.cumsum()
complete_grouped

,year_construction_completion,affordable_units,affordable_units_cumsum
0,2007,7,7
1,2013,129,136
2,2014,835,971
3,2015,419,"1,390"
4,2016,460,"1,850"
5,2017,"2,189","4,039"
6,2018,"2,134","6,173"
7,2019,"2,301","8,474"
8,2020,"1,044","9,518"
9,2021,"1,018","10,536"


In [851]:
# Here I create a cumulative chart

alt.Chart(complete_grouped).mark_area().encode(x='year_construction_completion', y='affordable_units_cumsum').properties(width=600, title='Affordable Unites Completed Over Time')

alt.Chart(...)

## B. Construction Time Analysis

In this part of our notebook we are going to analyze the average time required to complete a housing project.

Key data points: 
   * The average construction time per project is **1018** days. This matches with the average time spent on already completed constructions 
   * **929** days is the average time required to complete a project before the pandemic.
   * **1349** days is the average time required to complete a project after the pandemic.

Key findings:
   * Projects completed after the advent of the pandemic required an average of **420** more days of construction than those completed previously. 


In [852]:
# Convert required columns into datetime type

housing['issuance_of_building_permit'] = pd.to_datetime(housing['issuance_of_building_permit'])
housing['estimated_construction_completion'] = pd.to_datetime(housing['estimated_construction_completion'])
housing.head(3)

,project_id,project_status,project_name,street_number_,street_name,street_type,zip_code,block,lot,supervisor_district,city_analysis_neighborhood,planning_neighborhood,lead_agency,program_area,project_area,project_type,housing_tenure,ssp_acquisition_date,issuance_of_notice_to_proceed,issuance_of_building_permit,issuance_of_first_construction_document,estimated_actual_construction_start_date,estimated_construction_completion,project_lead_sponsor,project_co-sponsor,project_owner,planning__case_number,property_informaiton_map_link,planning_entitlements,entitlement_approval,section_415_declaration,building_permit_number,project_units,affordable_units,market_rate_units,per_affordable,sro_units,studio_units,1bd_units,2bd_units,3bd_units,4bd_units,5+_bd_units,mobility_units,manager_units,manager_unit(s)_type,family_units,senior_units,tay_units,homeless_units,disabled_units,losp_units,public_housing_replacement_units,20per_ami,30per_ami,40per_ami,50per_ami,55per_ami,60per_ami,80per_ami,90per_ami,100per_ami,105per_ami,110per_ami,120per_ami,130per_ami,150per_ami,ami_undeclared,latitude,longitude,location,year_construction_start,year_construction_completion
0,2008-009,complete,"HPSY I, Block 50",547,Donahue,St,94124,4591C,"265, 266",10,bayview_hunters_point,Bayview,OCII,Limited Equity Program,HP Shipyard Phase 1,New Construction,Ownership,NaN,NaN,2009-12-29,05/10/2010,05/10/2010,2016-04-27,Lennar Urban,NaN,NaN,Not Applicable,NaN,NaN,NaN,NaN,200907223200,25,3,22,12,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.728381185, -122.370199597999)",2010,2016
1,2013-044,complete,Mission Dolores,1855,15th,St,94103,3556,276,8,mission,Mission,MOHCD,Rental Assistance Demonstration Program (RAD),NaN,Rehabilitation,Rental,10/12/2016,11/07/2016,2016-05-16,11/03/2016,11/03/2016,2018-09-19,BRIDGE Housing Corporation,Mission Economic Development Agency,"Bridge Housing Ventures, Inc, MEDA Housing LLC",2015-012300PRJ,http://propertymap.sfplanning.org/?search=2015...,2015-012300ENV,NaN,NaN,201510301342,91,90,1,99,NaN,68,22,NaN,NaN,NaN,NaN,11,1,1bd,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.766017308, -122.425573258999)",2016,2018
2,2014-032,complete,Britton Courts (1250 Sunnydale Ave),171,Loehr,St,94134,6300,43,10,visitacion_valley,Visitacion Valley,MOHCD,Bond Financed Multifamily Housing,NaN,Rehabilitation,Rental,NaN,12/01/2017,NaT,NaN,12/01/2017,2018-12-01,Mercy Housing California,NaN,"Mercy Housing California 74, LP",Not Applicable,NaN,NaN,NaN,NaN,NaN,92,91,1,99,NaN,NaN,NaN,62,21,8,NaN,NaN,1,2bd,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,NaN,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.711716304, -122.413358024999)",2017,2018


In [853]:
# Here I create a new colum with the estimated construction time for each project

housing['time_construction'] = housing['estimated_construction_completion'] - housing['issuance_of_building_permit']
housing.head(3)

,project_id,project_status,project_name,street_number_,street_name,street_type,zip_code,block,lot,supervisor_district,city_analysis_neighborhood,planning_neighborhood,lead_agency,program_area,project_area,project_type,housing_tenure,ssp_acquisition_date,issuance_of_notice_to_proceed,issuance_of_building_permit,issuance_of_first_construction_document,estimated_actual_construction_start_date,estimated_construction_completion,project_lead_sponsor,project_co-sponsor,project_owner,planning__case_number,property_informaiton_map_link,planning_entitlements,entitlement_approval,section_415_declaration,building_permit_number,project_units,affordable_units,market_rate_units,per_affordable,sro_units,studio_units,1bd_units,2bd_units,3bd_units,4bd_units,5+_bd_units,mobility_units,manager_units,manager_unit(s)_type,family_units,senior_units,tay_units,homeless_units,disabled_units,losp_units,public_housing_replacement_units,20per_ami,30per_ami,40per_ami,50per_ami,55per_ami,60per_ami,80per_ami,90per_ami,100per_ami,105per_ami,110per_ami,120per_ami,130per_ami,150per_ami,ami_undeclared,latitude,longitude,location,year_construction_start,year_construction_completion,time_construction
0,2008-009,complete,"HPSY I, Block 50",547,Donahue,St,94124,4591C,"265, 266",10,bayview_hunters_point,Bayview,OCII,Limited Equity Program,HP Shipyard Phase 1,New Construction,Ownership,NaN,NaN,2009-12-29,05/10/2010,05/10/2010,2016-04-27,Lennar Urban,NaN,NaN,Not Applicable,NaN,NaN,NaN,NaN,200907223200,25,3,22,12,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.728381185, -122.370199597999)",2010,2016,2311 days
1,2013-044,complete,Mission Dolores,1855,15th,St,94103,3556,276,8,mission,Mission,MOHCD,Rental Assistance Demonstration Program (RAD),NaN,Rehabilitation,Rental,10/12/2016,11/07/2016,2016-05-16,11/03/2016,11/03/2016,2018-09-19,BRIDGE Housing Corporation,Mission Economic Development Agency,"Bridge Housing Ventures, Inc, MEDA Housing LLC",2015-012300PRJ,http://propertymap.sfplanning.org/?search=2015...,2015-012300ENV,NaN,NaN,201510301342,91,90,1,99,NaN,68,22,NaN,NaN,NaN,NaN,11,1,1bd,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.766017308, -122.425573258999)",2016,2018,856 days
2,2014-032,complete,Britton Courts (1250 Sunnydale Ave),171,Loehr,St,94134,6300,43,10,visitacion_valley,Visitacion Valley,MOHCD,Bond Financed Multifamily Housing,NaN,Rehabilitation,Rental,NaN,12/01/2017,NaT,NaN,12/01/2017,2018-12-01,Mercy Housing California,NaN,"Mercy Housing California 74, LP",Not Applicable,NaN,NaN,NaN,NaN,NaN,92,91,1,99,NaN,NaN,NaN,62,21,8,NaN,NaN,1,2bd,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,NaN,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,-122,"(37.711716304, -122.413358024999)",2017,2018,NaT


In [854]:
# Checking for errors

housing.sort_values('estimated_construction_completion').head().time_construction

161         NaT
466         NaT
405   1073 days
452    519 days
414   1740 days
Name: time_construction, dtype: timedelta64[ns]

With this step, we have found that in two rows, the `estimated_construction_completion` is prior to `estimateda_actual_construction_start_date`. 

To avoid any potential error, here we create a filter that states that the estimated construction completion time must be greater than the issuance of building permit time.

In [855]:
# Filter with condition 

housing = housing[housing.estimated_construction_completion > housing.issuance_of_building_permit]

Now, we are going to calculate the average construction time for **all projects**.

In [856]:
# Calculate the average construction time 

construction_time_average = housing['time_construction'].mean()
print(construction_time_average)

1018 days 04:32:39.763313616


Here, we are going to calculate the average construction time for **ongoing projects**.

In [857]:
# Calculate the average construction time: filter + calculation

filter_ongoing_projects = housing[housing.project_status.isin(['complete', 'cancelled']) == False]
average_time_ongoing_projects = filter_ongoing_projects['time_construction'].mean()
print(average_time_ongoing_projects)

1047 days 02:41:22.758620688


Here, we are going to check if the average construction time coincides with the time spent on already **completed constructions**.

In [858]:
# Filter + calculation

filter_completed_units = housing[housing['project_status'] == 'complete']
average_time_completed = filter_completed_units['time_construction'].mean()
print(average_time_completed)

1003 days 01:50:16.216216224


Here, we are going to check the difference between the average time required to complet a project and the average time of ongoing ones.

In [859]:
# Print calculation

print(average_time_completed - average_time_ongoing_projects)

-45 days +23:08:53.457595536


This could be a potential underestimation of the time needed for the construction.

#### Pandemic calculations:

Since this analysis focuses on construction time, we must take into account an exceptional situation that hit the sector hard: the covid-19 pandemic. Therefore, we will perform two different analyses pre-pandemic and post-pandemic:

Key findings: 

   * **929 days** is the average time required to complete a project **before the pandemic**.
   * **1349 days** is the average time required to complete a project **after the pandemic**.

In [860]:
# Pre-pandemic calculations
average_time_ongoing_projects_prepandemic = filter_ongoing_projects[filter_ongoing_projects.year_construction_completion < 2020]['time_construction'].mean()
average_time_completed_prepandemic = filter_completed_units[filter_completed_units.year_construction_completion < 2020]['time_construction'].mean()

print('Ongoing projects:', average_time_ongoing_projects_prepandemic)
print('Completed projects:', average_time_completed_prepandemic)
print('Ongoing - Completed = ', average_time_ongoing_projects_prepandemic - average_time_completed_prepandemic)

Ongoing projects: 1167 days 16:36:55.384615392
Completed projects: 949 days 21:02:01.348314608
Ongoing - Completed =  217 days 19:34:54.036300784


In [861]:
# Post-pandemic calculations
average_time_ongoing_projects_postpandemic = filter_ongoing_projects[filter_ongoing_projects.year_construction_completion > 2020]['time_construction'].mean()
average_time_completed_postpandemic = filter_completed_units[filter_completed_units.year_construction_completion > 2020]['time_construction'].mean()

print('Ongoing projects:', average_time_ongoing_projects_postpandemic)
print('Completed projects:', average_time_completed_postpandemic)
print('Ongoing - Completed = ', average_time_ongoing_projects_postpandemic - average_time_completed_postpandemic)

Ongoing projects: 1030 days 15:48:42.352941184
Completed projects: 1349 days 14:40:00
Ongoing - Completed =  -319 days +01:08:42.352941184


### A + B Finding:

The **Project Status Analysis** has allowed us to find that there is a construction backlog of nearly **2,000 affordable units**. The main backlog appears in 2021, with more than **1,300** unfinished affordable units. 

The **Construction Time Analysis** has allowed us to verify that there is a significant difference in the time required to build the projects before and after the pandemic. Projects completed since the outbreak of the pandemic have taken an average of **420** days longer to completion.  

In Nov. 2021, a workforce survey conducted by the Associated General Contractors of America and Autodesk, stated that "construction firms are struggling to find enough qualified workers to hire even as they continue to be impacted by pandemic-induced project delays and supply chain disruptions." In addition, the survey results "underscore how the coronavirus pandemic has created constraints on the demand for work even as it limits the number of workers available to hire." Source: The Associated General Contractors of America (url: https://www.agc.org/news/2021/09/02/construction-workforce-shortages-reach-pre-pandemic-levels-even-coronavirus).

Therefore, the delay in the construction of the affordable units is related to the pandemic, as we might expect.


## C. Geographical Analysis

In this third part of our analysis we will focus on the distribution of affordable housing by neighborhood in San Francisco.

For this purpose, we are goinf to use the following databases:
* SF Affordable Housing Pipeline Database. 
* The estimations that appear in the SF 2014 Housing Element (url: https://generalplan.sfplanning.org/2014HousingElement-AllParts_ADOPTED_web.pdf).

Data Limitations:
* The estimations that appear in the SF 2014 Housing Element only include 8 SF neighborhoods.
* These projections are limited to the period between 2014 and 2022. 

Key findings:
   * The neighborhoods where most construction has taken place in the last ten years are **South of Market** and **Financial District**, in these areas the percentage of **affordable units** does not exceed **26%**. 
   * **Mission** and **Bayview Hunters Point** neighborhoods have experienced a significant increase in the number of units built; in these cases, the percentage of affordable units is **45%** and **73%**, respectively. 
   * The SF city's forecast is far from the actual figures. The largest differences between the city's estimates and the actual figures appear in the following neighborhoods: **Bayview Hunters Point, South of Market and Financial District**.

In [862]:
# Check the values

housing.city_analysis_neighborhood.value_counts()

mission                           56
south_of_market                   52
tenderloin                        29
bayview_hunters_point             26
financial_district_south_beach    26
potrero_hill                      23
hayes_valley                      21
mission_bay                       13
western_addition                  13
marina                            10
nob_hill                           9
pacific_heights                    7
castro_upper_market                6
outer_mission                      5
sunset_parkside                    4
russian_hill                       4
chinatown                          4
outer_richmond                     3
lone_mountain_usf                  3
west_of_twin_peaks                 3
treasure_island                    3
oceanview_merced_ingleside         2
visitacion_valley                  2
portola                            2
north_beach                        2
inner_sunset                       2
bernal_heights                     2
p


#### Analysis per Neighborhood:

In this part of the analysis we will focus only on the affordable units that have already been **completed**. 

We are going to limit the time frame to the last 10 years: **2012-2022**. 

In this section we only use the **SF Affordable Housing Pipeline Database**.

In [863]:
# Create a filter with project_status + time frame

complete_all = housing[(housing['project_status'] == 'complete') & (housing.year_construction_completion > 2011) & (housing.year_construction_completion < 2023)]

Here, we are going to create a new table with the requiered fields: `project_units` and `affordable_units`.

In [864]:
# Create a new table with completed units -affordable and non affordable– by neighborhood 

grouped_complete = complete_all.groupby('city_analysis_neighborhood')[['project_units', 'affordable_units']].sum().reset_index()
grouped_complete.head(3)

,city_analysis_neighborhood,project_units,affordable_units
0,bayview_hunters_point,"1,982","1,450"
1,bernal_heights,268,265
2,castro_upper_market,364,145


Next, we are going to create a new column with subtraction of `project_units` minus `affordable_units`. This step is necessary to be able to configure our next graph. 

In [865]:
# Create a new column with the subtraction of "project units" minus "affordable units".

grouped_complete['other_units'] = grouped_complete.project_units - grouped_complete.affordable_units
grouped_complete.head(3)

,city_analysis_neighborhood,project_units,affordable_units,other_units
0,bayview_hunters_point,"1,982","1,450",532
1,bernal_heights,268,265,3
2,castro_upper_market,364,145,219


Here, we continue to proceed with the configuration of our table in order to make the following graph. 

In [866]:
# Melt the data

melted_complete = grouped_complete.melt(id_vars=['city_analysis_neighborhood', 'project_units'], var_name='category', value_name='units')

In [867]:
# Create new columns with the % of affordable –in values and text–. 

melted_complete['pct_affordable'] = np.where(melted_complete.category == 'affordable_units', 
                                            (melted_complete.units / melted_complete.project_units * 100), None)
melted_complete['pct_affordable_text'] = np.where(melted_complete.category == 'affordable_units', 
                                                  (melted_complete.units / melted_complete.project_units * 100).apply(lambda x: str(round(x)) + '%'),
                                                 '')
melted_complete.head(3)

,city_analysis_neighborhood,project_units,category,units,pct_affordable,pct_affordable_text
0,bayview_hunters_point,"1,982",affordable_units,"1,450",73,73%
1,bernal_heights,268,affordable_units,265,99,99%
2,castro_upper_market,364,affordable_units,145,40,40%


Now that our table is ready, we proceed to make the graph: **"Total Units Completed by Neighboord Between 2012 and 2022."**

In [868]:
# Create a chart with the total number of units and affordable units ordered by neigborhood with the largest number of units completed

bars = alt.Chart(melted_complete).mark_bar().encode(x='units', y=alt.Y('city_analysis_neighborhood', 
                                                                      sort=alt.EncodingSortField(field="units", op="sum", order="descending")), 
                                                    color='category')

affordable_text = bars.mark_text(stroke='steelblue', align='left', dx=5).encode(
    text='pct_affordable_text'
)

(bars+affordable_text).properties(width=900, title='Total Units Completed by Neighboord Between 2012 and 2022')

alt.LayerChart(...)

As we can see, the neighborhoods where most of the construction has taken place are: **South of Market**, **Financial District** and **Mission**. However, in the first two neighborhoods mentioned, the number of **affordable housing** units does not exceed **26%**. 
This percentage is much more significant in **Mission** and **Bayview Hunters Point** neighborhoods, where the percentage of affordable housing is higher: **45%** and **73%**, respectively.

Next, we are going to make the same graph but ordering it from **highest to lowest** according to the **percentage of completed affordable units**. 

In [869]:
# Create a new char ordered by the neighborhoods with the largest number of affordable units completed

bars = alt.Chart(melted_complete).mark_bar().encode(x='units', y=alt.Y('city_analysis_neighborhood', 
                                                                      sort=alt.EncodingSortField(field="pct_affordable", op="sum", order="descending")), 
                                                    color='category')

affordable_text = bars.mark_text(stroke='steelblue', align='left', dx=5).encode(
    text='pct_affordable_text'
)

(bars+affordable_text).properties(width=900, title='Affordable Units Completed by Neighborhood Between 2012 and 2022')

alt.LayerChart(...)

As we can see, in some neighborhoods, almost all of the units built are affordable, although the total number of units built is very low.

#### Comparison of the actual numbers with the city's expectations: 

In order to perform our next analysis, we need to shift the time frame of our analysis to **2014-2022**. 

Here we are going to use both databases: **SF Affordable Housing Pipeline Database** + **SF 2014 Housing Element**.

In [870]:
# Filter the data and reset table.

complete_new_time_frame = housing[(housing['project_status'] == 'complete') & (housing.year_construction_completion > 2013) & (housing.year_construction_completion < 2023)]
affordable_per_area = complete_new_time_frame.groupby('city_analysis_neighborhood').agg({'project_units': 'sum','affordable_units':'sum', 'per_affordable': 'median'}).reset_index()
affordable_per_area.sort_values(['per_affordable'], ascending=False)

,city_analysis_neighborhood,project_units,affordable_units,per_affordable
3,chinatown,545,523,100
0,bayview_hunters_point,"1,982","1,450",99
9,japantown,135,134,99
27,twin_peaks,109,108,99
26,tenderloin,"1,589",970,99
1,bernal_heights,268,265,99
14,mission_bay,"1,757",904,99
30,western_addition,"1,171",878,99
7,inner_richmond,69,68,99
16,north_beach,116,114,98


Now, we are going to import our second dataset: **SF Affordable Plan 2014-2022**, from a locally saved copy. 

In [871]:
# Import database

units_file = os.path.join(raw_data, 'affordable_plan_sf.csv')
units_expected = pd.read_csv(units_file)

Here, we are **vetting** the new added dataset:

In [872]:
units_expected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   city_analysis_neighborhood      17 non-null     object
 1   planning_neighborhood_original  17 non-null     object
 2   adopted_plans_projects          17 non-null     int64 
 3   plans_projects_underway         17 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 672.0+ bytes


In [873]:
units_expected.describe()

,adopted_plans_projects,plans_projects_underway
count,17,17
mean,"2,431","1,129"
std,"2,885","2,305"
min,0,0
25%,0,0
50%,"1,700",0
75%,"3,200","1,200"
max,"10,500","8,000"


Now, we are going to calculate the total number of expected units: `adopted_plans_projects` + `plans_projects_underway`

In [874]:
# Create a column with total units expected

units_expected["total_units_expected"] = units_expected["adopted_plans_projects"] + units_expected["plans_projects_underway"]

Next, we are going to create a new table grouped by `city_analysis_neighborhood` with the `total_units_expected`

In [875]:
# Group by neighborhood

affordable_units_expected_grouped = units_expected.groupby('city_analysis_neighborhood').total_units_expected.sum().reset_index()

To continue our analysis, here we need to **merge** both databases. 

In [876]:
# Merging

merged = affordable_per_area.merge(affordable_units_expected_grouped)

**According to the 2014 SF Housing Element**: "In an effort to manage the regional growth and accommodate projected housing needs throughout the Bay Area, the Association of Bay Area Governments (ABAG) allocates a number of housing units at various income levels to each community in the region based on projected job growth. ABAG has allocated about 29,000 new housing units in City and County of San Francisco through the year 2022, with over **57%** of those units required to be affordable to households of moderate income (defined as 120% of Area Median Income) or below."

29,000 new housing units is the lowest estimation of the city. Inside the 2014 SF Housing Element, we found a table with additional projections. To perform our analysis we are going to use these projections and calculate the minimum number of affordable units by applying the 57% rule. 

In [877]:
# Create a new colum with affordable united expected – From total: "over 57% of those units required to be affordable to households of moderate income."

merged["minimum_affordable_units_expected"] = 57 * merged["total_units_expected"] / 100
merged

,city_analysis_neighborhood,project_units,affordable_units,per_affordable,total_units_expected,minimum_affordable_units_expected
0,bayview_hunters_point,"1,982","1,450",99,12100,"6,897"
1,financial_district_south_beach,"2,985",640,14,8640,"4,925"
2,mission,"2,162",963,16,3500,"1,995"
3,mission_bay,"1,757",904,99,2000,"1,140"
4,south_of_market,"3,758",993,12,9700,"5,529"
5,tenderloin,"1,589",970,99,6000,"3,420"
6,twin_peaks,109,108,99,100,57
7,visitacion_valley,55,54,98,1680,958


In [878]:
# Here I melt the data, this step is necessary in order to build the next chart -difference between expected and reality-. 

melted = merged[['city_analysis_neighborhood', 'affordable_units', 'minimum_affordable_units_expected']].melt(id_vars='city_analysis_neighborhood')
melted

,city_analysis_neighborhood,variable,value
0,bayview_hunters_point,affordable_units,"1,450"
1,financial_district_south_beach,affordable_units,640
2,mission,affordable_units,963
3,mission_bay,affordable_units,904
4,south_of_market,affordable_units,993
5,tenderloin,affordable_units,970
6,twin_peaks,affordable_units,108
7,visitacion_valley,affordable_units,54
8,bayview_hunters_point,minimum_affordable_units_expected,"6,897"
9,financial_district_south_beach,minimum_affordable_units_expected,"4,925"


In [879]:
# Create a chart

lines = alt.Chart(melted).mark_line(color='grey').encode(
    x='value',
    y='city_analysis_neighborhood',
    detail='city_analysis_neighborhood'
)

points = alt.Chart(melted).mark_point(size=50,
    opacity=1,
    filled=True).encode(
    x='value',
    y='city_analysis_neighborhood',
    color=alt.Color('variable:O',
        scale=alt.Scale(
            domain=['affordable_units', 'minimum_affordable_units_expected'],
            range=['#e6959c', '#911a24']
        ))
)

(lines+points).properties(width=600, title='SF is far from its Affordable Housing Estimations ')

alt.LayerChart(...)

As we can see, the city's forecast is far from the actual figures. The largest differences between the city's estimates and the actual figures appear in the following neighborhoods: **Bayview Hunters Point, South of Market and Financial District**.

### Finding C:

After an extensive analysis of construction by neighborhood, we found that the neighborhoods where the most construction took place between 2014 and 2022 were **South of Market** and **Financial District**, however, these housing projects did not lead to a significant increase in the number of affordable units.